In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import sys

In [ ]:
# !pip download scikit-maad==1.4.1 -d /kaggle/working/mysitepackages/sklearn_maad_pkg
#!pip download ffmpeg-python==0.2.0 -d /kaggle/working/mysitepackages/ffmpeg_pkg

In [ ]:
"""from zipfile import ZipFile
import os

dirName = '/kaggle/working/mysitepackages'
zipName = 'ffmpeg_pkg.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

In [ ]:
#!pip install ffmpeg-python --no-index --find-links=file:///kaggle/input/ffmpeg-pkg/kaggle/working
#!pip install scikit-maad --no-index --find-links=file:///kaggle/input/sklearn-maad-offline/kaggle/working/mysitepackages/sklearn_maad_pkg

In [ ]:
"""from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from maad import sound, rois, features
from maad.util import (
   power2dB, plot2d, format_features, read_audacity_annot,
   overlay_rois, overlay_centroid
   )"""

In [12]:
config = {}
Test = False

if Test:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/'
    config['start'] = 7000
    config['end'] = 8200
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path][config['start']:config['end']]
    print(len(filenames))
else:
    config['root_file_path'] = '/kaggle/input/birdclef-2024/test_soundscapes/'
    filenames_with_path = glob.glob(f"{config['root_file_path']}*.ogg")
    filenames = [os.path.basename(filename) for filename in filenames_with_path]
    print(len(filenames))

In [14]:
chunk_feature_dict = {}

for file in tqdm(filenames):
    full_path = config['root_file_path']+file
    audio, sample_rate = librosa.load(path=full_path, sr=32000)
    cur_file = file.replace('.ogg','')
    samples_per_segment = sample_rate * 5
    if len(audio) != 7680000:
        total_samples = 7680000
    else:
        total_samples = len(audio)
        
    for i in range(0, total_samples+160000, samples_per_segment):
        if i + samples_per_segment <= total_samples:
            segment = audio[i:i + samples_per_segment]
            chunk = cur_file+'_'+str(int((i/32000)+5))
            mfccs = librosa.feature.mfcc(y=segment, sr=32000, n_mfcc=40)
            flattened_features = (np.mean(mfccs.T, axis=0)).reshape(1, -1)
            chunk_feature_dict[chunk] = flattened_features

0it [00:00, ?it/s]


In [15]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)
bird_cols = list(meta_data['primary_label'].unique().sort())

submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit['row_id'] = 'samples'

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
1,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495
2,samples,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,...,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495,0.005495


In [6]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [16]:
# this is where the inference loop will go

for file_name, feature_set in tqdm(chunk_feature_dict.items()):
    df = pd.DataFrame(random_forest_model.predict_proba(feature_set), columns=bird_cols)
    df.insert(loc=0, column='row_id', value=file_name)
    submit = pd.concat([submit,df]).reset_index(drop=True)
    
i = submit[(submit.row_id == 'samples')].index
submit = submit.drop(i).reset_index(drop=True)

del chunk_feature_dict

0it [00:00, ?it/s]


In [8]:
#submit.sum(axis=1, numeric_only=True)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
57595    1.0
57596    1.0
57597    1.0
57598    1.0
57599    1.0
Length: 57600, dtype: float64

In [10]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submit.columns)

In [11]:
submit.to_csv('submission.csv', index=False)